# DuckDB and DyalogAPL with Vega-Lite

In [1]:
⎕pw←32767 ⋄ ⎕PP←34

In [2]:
]link.create # ../dyalog-jupyter-vegalite

Linked: # ←→ /home/kimmo/dyalog-jupyter-vegalite/../dyalog-jupyter-vegalite

In [3]:
duckdb.init 'lib/'
_db←duckdb.open ':memory:'
_con←duckdb.connect _db
_con

94063429625264

In [4]:
duckdb.query _con 'CREATE TABLE elec AS SELECT * FROM "data/electricity.parquet"'

┌───────┬─────┬─┐
│┌─────┐│11758│5│
││Count││ │ │
│└─────┘│ │ │
└───────┴─────┴─┘

In [5]:
duckdb.toTable duckdb.query _con 'PRAGMA table_info(elec)'

┌───┬───────────┬─────────┬───────┬──────────┬──┐
│cid│name │type │notnull│dflt_value│pk│
├───┼───────────┼─────────┼───────┼──────────┼──┤
│0 │time │TIMESTAMP│0 │ [Null] │0 │
├───┼───────────┼─────────┼───────┼──────────┼──┤
│1 │consumption│DOUBLE │0 │ [Null] │0 │
├───┼───────────┼─────────┼───────┼──────────┼──┤
│2 │price │DOUBLE │0 │ [Null] │0 │
└───┴───────────┴─────────┴───────┴──────────┴──┘

In [6]:
duckdb.toTable duckdb.query _con 'SELECT * FROM elec ORDER BY time LIMIT 5'

┌───────────────────┬───────────┬─────┐
│time │consumption│price│
├───────────────────┼───────────┼─────┤
│2020 12 31 23 0 0 0│4.6 │24.95│
├───────────────────┼───────────┼─────┤
│2021 1 1 0 0 0 0 │9.27 │24.35│
├───────────────────┼───────────┼─────┤
│2021 1 1 1 0 0 0 │9.4 │23.98│
├───────────────────┼───────────┼─────┤
│2021 1 1 2 0 0 0 │9.31 │23.72│
├───────────────────┼───────────┼─────┤
│2021 1 1 3 0 0 0 │4.89 │23.73│
└───────────────────┴───────────┴─────┘

In [7]:
]multiline
duckdb.toTable duckdb.query _con ```
SELECT 
    year(time) as year,
    month(time) as month,
    sum(consumption) as consumption,
    sum(1.24*price*consumption/1000) as costs,
    avg(1.24*price/10) as avg_price,
    sum(1.24*price/10*consumption)/sum(consumption) as weighted_price 
FROM elec
WHERE year(time)>2020 
GROUP BY year(time),month(time)
```

┌────┬─────┬───────────┬───────────┬───────────┬──────────────┐
│year│month│consumption│costs │avg_price │weighted_price│
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2021│1 │3530.99 │215.9300066│6.354555 │6.115282303 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2021│2 │3384.76 │239.1570573│7.079901786│7.065702068 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2021│3 │2945.29 │131.5138099│4.751558333│4.465224473 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2021│4 │2329.58 │100.6547046│4.567354 │4.320723248 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2021│5 │1948.1 │100.6957292│5.69209 │5.168919931 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2021│6 │1797.95 │111.9910938│6.965927333│6.228821367 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2021│7 │1681.7 │153.0326578│9.77001 │9.099878563 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2021│8 │1721.89 │136.432141 │8.44901 │7.923394703 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2021│9 │1884.44 │198.4407274│11.07337222│10.53048797 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2021│10 │2433.58 │180.1846079│8.041026667│7.40409635 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2021│11 │3154.2 │340.4261995│10.67069944│10.79279055 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2021│12 │4709.34 │1175.390424│23.9415505 │24.95870809 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│1 │4044.6 │525.8049314│13.16062441│13.00017137 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│2 │3078.02 │306.7896531│10.0258244 │9.967110452 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│3 │2777.89 │286.8994659│10.74263833│10.32796352 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│4 │2145.95 │195.0728148│9.8475375 │9.090277721 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│5 │218.15 │30.27712799│15.16070667│13.87904102 │
└────┴─────┴───────────┴───────────┴───────────┴──────────────┘

In [8]:
duckdb.query _con 'SELECT strftime(time,''%Y-%m-%dT%H:%MZ'') as date,price as value FROM elec LIMIT 5'

┌────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────┐
│┌────┬─────┐│┌───────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────────┐│17 11│
││date│value│││┌─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┐│24.95 24.35 23.98 23.72 23.73││ │
│└────┴─────┘│││2020-12-31T23:00Z│2021-01-01T00:00Z│2021-01-01T01:00Z│2021-01-01T02:00Z│2021-01-01T03:00Z││ ││ │
│ ││└─────────────────┴─────────────────┴─────────────────┴─────────────────┴─────────────────┘│ ││ │
│ │└───────────────────────────────────────────────────────────────────────────────────────────┴─────────────────────────────┘│ │
└────────────┴───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┴─────┘

```apl
json←toJson in
col data←2↑in
data←{a←⍵ ⋄ ∨/b←⍵∊⎕null:{(b/a)←⍵⋄a}⊂'null' ⋄ ⍵}¨data
json←⎕json ⊂ 4 (data col)

⍝ KSL 1.0.2
```

In [9]:
]multiline
cons_daily←duckdb.toJson duckdb.query _con ```
SELECT
  strftime(date_trunc(''day'',time),''%Y-%m-%d'') as date,
  round(sum(consumption),2) as cons_daily
FROM elec
WHERE year(time)>2020
GROUP BY date_trunc(''day'',time)
ORDER BY date
``` ⋄ 100↑cons_daily

[{"date":"2021-01-01","cons_daily":95.08},{"date":"2021-01-02","cons_daily":122.41},{"date":"2021-01

In [10]:
]multiline
cons_monthly←duckdb.toJson duckdb.query _con ```
SELECT
  strftime(date_trunc(''month'',time),''%Y-%m-%d'')as month,
  round(sum(consumption),2) as cons_monthly
FROM elec
WHERE year(time)>2020
GROUP BY date_trunc(''month'',time)
ORDER BY month
``` ⋄ 100↑cons_monthly

[{"month":"2021-01-01","cons_monthly":3530.99},{"month":"2021-02-01","cons_monthly":3384.76},{"month

In [11]:
]multiline
price_daily←duckdb.toJson duckdb.query _con ```
SELECT
  strftime(date_trunc(''day'',time),''%Y-%m-%d'') as date,
  min(price) as min,
  max(price) as max
FROM elec
WHERE year(time)>2020
GROUP BY date_trunc(''day'',time)
ORDER BY date
``` ⋄ 100↑price_daily

[{"date":"2021-01-01","min":23.72,"max":30.25},{"date":"2021-01-02","min":24.66,"max":60.51},{"date"

In [12]:
cons_daily ⎕nput 'data/cons_daily.json' 1
cons_monthly ⎕nput 'data/cons_monthly.json' 1
price_daily ⎕nput 'data/price_daily.json' 1

In [13]:
]vegalite
'plots/layerplot.vl'layerPlot ('data/cons_daily.json' 'data/cons_monthly.json')

In [15]:
]vegalite
'plots/lineplot.vl'linePlot ('data/price_daily.json')

In [16]:
_←duckdb.disconnect _con
_←duckdb.close _db 

{
 "mark": {
 "type": "area"
 },
 "data": {
 "url": "data/price_daily.json"
 },
 "encoding": {
 "x": {
 "field": "date",
 "type": "temporal"
 },
 "y2": {
 "field": "max",
 "type": "quantitative"
 },
 "y": {
 "field": "min",
 "type": "quantitative"
 }
 },
 "width": 800,
 "height": 400
}